## Evaluation of the YOLOv8 on COCO images:
*Does the model accurately detect all cars in our data set or there is room for improvement? Evaluate the model on COCO images with cars, Why do you think the model does perform equally on our data?*



Install dependencies:

In [ ]:
# Install YOLOv8 and pycocotools
!pip install ultralytics pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.3 MB/s eta 0:00:00


Download validation and annotation images, and unzip them:

In [ ]:
# Download validation images
!wget http://images.cocodataset.org/zips/val2017.zip
# Download annotations
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

import zipfile
import os

val_zip = "/content/val2017.zip"
extract_path = "/content/"

with zipfile.ZipFile(val_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Validation images extracted to: {extract_path}val2017/")


ann_zip = "/content/annotations_trainval2017.zip"
ann_extract_path = "/content/"

with zipfile.ZipFile(ann_zip, 'r') as zip_ref:
    zip_ref.extractall(ann_extract_path)

print(f"Annotations extracted to: {ann_extract_path}")

--2025-09-30 11:49:55--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.164.81, 16.182.39.145, 3.5.29.244, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.164.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  16.6MB/s    in 49s     

2025-09-30 11:50:45 (15.8 MB/s) - ‘val2017.zip’ saved [815585330/815585330]

--2025-09-30 11:50:45--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.224.217, 52.217.170.25, 54.231.195.185, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.224.217|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_train

Load the model:

In [ ]:
from ultralytics import YOLO

# Load trained YOLOv8 model
model = YOLO("yolov8n.pt")

Load COCO annotations and get images containing cars:

In [ ]:
from pycocotools.coco import COCO
import os
import shutil

# Path to annotations and images
ann_file = "annotations/instances_val2017.json"
image_folder = "val2017"

coco = COCO(ann_file)
car_cat_id = 3
img_ids = coco.getImgIds(catIds=[car_cat_id])
images = coco.loadImgs(img_ids)

# Copy only car images to a separate folder
car_image_folder = "coco_car_images/"
os.makedirs(car_image_folder, exist_ok=True)

for img in images:
    src = os.path.join(image_folder, img['file_name'])
    dst = os.path.join(car_image_folder, img['file_name'])
    if not os.path.exists(dst):
        shutil.copy(src, dst)

print(f"Copied {len(images)} car images to {car_image_folder}")

loading annotations into memory...
Done (t=0.80s)
creating index...
index created!
Copied 535 car images to coco_car_images/


Run YOLOv8 inference and create preds_json (COCO format)

In [ ]:
import json
preds_coco = []

for img in images:
    img_path = os.path.join(car_image_folder, img['file_name'])
    results = model(img_path)[0]  # get YOLOv8 predictions

    boxes = results.boxes.xyxy  # x1,y1,x2,y2
    scores = results.boxes.conf
    classes = results.boxes.cls

    for box, score, cls in zip(boxes, scores, classes):
        # Only keep predictions of class 'car' (depends on your model class IDs)
        if int(cls) == 2:  # car ID = 2
            x1, y1, x2, y2 = box.tolist()
            bbox = [x1, y1, x2 - x1, y2 - y1]  # convert to COCO format [x, y, w, h]
            pred_dict = {
                "image_id": img['id'],
                "category_id": 3,
                "bbox": bbox,
                "score": float(score)
            }
            preds_coco.append(pred_dict)

# Save predictions to JSON
preds_json_path = "/content/yolo_preds_on_coco_car.json"
with open(preds_json_path, "w") as f:
    json.dump(preds_coco, f)

print(f"Saved predictions to {preds_json_path}")



image 1/1 /content/coco_car_images/000000532481.jpg: 448x640 2 persons, 1 car, 1 snowboard, 1 kite, 122.8ms
Speed: 6.9ms preprocess, 122.8ms inference, 510.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/coco_car_images/000000184324.jpg: 448x640 14 persons, 2 bicycles, 7 cars, 2 motorcycles, 1 truck, 1 handbag, 17.5ms
Speed: 3.7ms preprocess, 17.5ms inference, 28.9ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/coco_car_images/000000454661.jpg: 448x640 5 cars, 3 traffic lights, 13.1ms
Speed: 2.4ms preprocess, 13.1ms inference, 8.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/coco_car_images/000000546823.jpg: 480x640 14 persons, 1 dog, 1 horse, 3 umbrellas, 87.1ms
Speed: 2.7ms preprocess, 87.1ms inference, 17.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/coco_car_images/000000393226.jpg: 480x640 1 person, 4 cars, 2 trucks, 1 traffic light, 13.8ms
Speed: 2.3ms preprocess, 13.8ms inference, 

Evaluate with COCOeval

In [ ]:
from pycocotools.cocoeval import COCOeval

# Load predictions
cocoDt = coco.loadRes(preds_json_path)

# Obtain IDs from car images
imgIds = [img['id'] for img in images]

# Prepare evaluation for bounding boxes
cocoEval = COCOeval(coco, cocoDt, iouType='bbox')
# Restrict evaluation to images with cars
cocoEval.params.imgIds = imgIds
# Restrict evaluation to car category only
cocoEval.params.catIds = [3]
# Compare predicted boxes with ground truth
cocoEval.evaluate()
# Aggregate results
cocoEval.accumulate()
# Show metrics
cocoEval.summarize()

Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.58s).
Accumulating evaluation results...
DONE (t=0.08s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.323
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.477
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.354
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.186
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.527
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.614
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.155
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.364
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.370
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=10